In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import csv
from textblob import TextBlob
import pandas
import sklearn
import pickle
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split 
from sklearn.tree import DecisionTreeClassifier 
from sklearn.model_selection import learning_curve

In [9]:
! git init 
! dvc init -f

hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
Initialized empty Git repository in /home/jexica/Applied-ML-course/Assignment2/.git/


Initialized DVC repository.

You can now commit the changes to git.

+---------------------------------------------------------------------+
|                                                                     |
|        DVC has enabled anonymous aggregate usage analytics.         |
|     Read the analytics documentation (and how to opt-out) here:     |
|             <https://dvc.org/doc/user-guide/analytics>              |
|                                                                     |
+---------------------------------------------------------------------+

What's next?
------------
- Check out the documentation: <https://dvc.org/doc>
- Get help and share ideas: <https://dvc.org/chat>
- Star us on GitHub: <https://github.com/iterative/dvc>


In [13]:
! dvc remote add -d remote_storage gdrive://1mBSh692-OHn7R_tIz-j2QxLp6AKN91Jm
! git log
! dvc add "data/raw"
! dvc add "data/prepared"
! git add .
! git commit -m "Raw data & Initialize DVC"

Setting 'remote_storage' as a default remote.
commit 256239e829ff3061bd09f87b006d5ad7fc5871f7 (HEAD -> second_try, origin/main, origin/HEAD, main)
Author: JESSICA VIPIN <73703357+jessicavipin@users.noreply.github.com>
Date:   Tue Feb 20 23:45:05 2024 +0800

    Add files via upload

commit 54825859bdda6d4b8368b132690d8912cacead84
Author: JESSICA VIPIN <73703357+jessicavipin@users.noreply.github.com>
Date:   Wed Feb 7 05:48:22 2024 +0800

    Update README.md

commit c00140f7f1e4bdd439f6d391b5d13871189b65d0
Author: JESSICA VIPIN <73703357+jessicavipin@users.noreply.github.com>
Date:   Wed Feb 7 05:47:29 2024 +0800

    Update README.md

commit 033c20529c0bfe9a03208d1e0bcc2768dc6324c4
Author: JESSICA VIPIN <73703357+jessicavipin@users.noreply.github.com>
Date:   Wed Feb 7 05:47:07 2024 +0800

    Update README.md

commit 1a6e9b65c29eef38d261b97ad9e9241b8acd6c04
Author: JESSICA VIPIN <73703357+jessicavipin@users.noreply.github.com>
Date:   Wed Feb 7 05:45:56 2024 +0800

    Update README.

In [12]:
! dvc add "data/raw"

⠋ Checking graph                                       core>
Adding...                                                                       
!
Adding...                                                                       
ERROR:  output 'data/raw' is already tracked by SCM (e.g. Git).
    You can remove it from Git, then add to DVC.
        To stop tracking from Git:
            git rm -r --cached 'data/raw'
            git commit -m "stop tracking data/raw" 


In [11]:
#load the data from a given file path
data = pandas.read_csv('data/raw/emails.csv',sep=',',header=None)
data= data[1:]
data.columns=["message", "label"]
print(data)

                                                message label
1     Subject: naturally irresistible your corporate...     1
2     Subject: the stock trading gunslinger  fanny i...     1
3     Subject: unbelievable new homes made easy  im ...     1
4     Subject: 4 color printing special  request add...     1
5     Subject: do not have money , get software cds ...     1
...                                                 ...   ...
5724  Subject: re : research and development charges...     0
5725  Subject: re : receipts from visit  jim ,  than...     0
5726  Subject: re : enron case study update  wow ! a...     0
5727  Subject: re : interest  david ,  please , call...     0
5728  Subject: news : aurora 5 . 2 update  aurora ve...     0

[5728 rows x 2 columns]


In [36]:
#Preprocess the data
def split_into_lemmas(data):
    data = data.lower()
    words = TextBlob(data).words
    # for each word, take its "base form" = lemma 
    return [word.lemma for word in words]

data.message.head().apply(split_into_lemmas)
print(data)

                                                message label
1     Subject: naturally irresistible your corporate...     1
2     Subject: the stock trading gunslinger  fanny i...     1
3     Subject: unbelievable new homes made easy  im ...     1
4     Subject: 4 color printing special  request add...     1
5     Subject: do not have money , get software cds ...     1
...                                                 ...   ...
5724  Subject: re : research and development charges...     0
5725  Subject: re : receipts from visit  jim ,  than...     0
5726  Subject: re : enron case study update  wow ! a...     0
5727  Subject: re : interest  david ,  please , call...     0
5728  Subject: news : aurora 5 . 2 update  aurora ve...     0

[5728 rows x 2 columns]


In [115]:
#First split the data into train/validation/test
train_val_data, test_data = train_test_split(data, test_size=0.2, random_state=42)
train_data, validation_data = train_test_split(train_val_data, test_size=0.25, random_state=42)  # 0.25 x 0.8 = 0.2
train_data.to_csv('data/prepared/train.csv', index=False)
validation_data.to_csv('data/prepared/validation.csv', index=False)
test_data.to_csv('data/prepared/test.csv', index=False)

In [38]:
! dvc status
! dvc add "data/raw/emails.csv"
! dvc add "data/prepared"

data/prepared.dvc:                                                              
	changed outs:
		modified:           data/prepared
⠋ Checking graph                                       core>
Adding...                                                                       
!
                                                                                
!
  0% Checking cache in '/home/jexica/Applied-ML-course/Assignment2/.dvc/cache/fi
                                                                                
!
  0%|          |Checking out /home/jexica/Applied-ML-c0/1 [00:00<?,    ?files/s]
100% Adding...|████████████████████████████████████████|1/1 [00:00, 16.50file/s]
                                                                core>⠋ Checking graph
Adding...                                                                       
!
                                                                                
!
  0% Checking cache in '/home/jexica/Applied-ML-course/Assignme

In [39]:
! dvc push
! git add .
! git commit -m "Adding the first version of split data"

Pushing
!
  0% Querying remote cache|                          |0/1 [00:00<?,    ?files/s]
100% Querying remote cache|█████████████████████|1/1 [00:09<00:00,  9.84s/files]
                                                                                
!
  0% Querying remote cache|                          |0/0 [00:00<?,    ?files/s]
                                                                                
!
  0% Checking cache in '1mBSh692-OHn7R_tIz-j2QxLp6AKN91Jm/files/md5'| |0/? [00:0
100% Querying cache in '1mBSh692-OHn7R_tIz-j2QxLp6AKN91Jm/files/md5'|█|1/1 [00:0
Pushing                                                                         
Everything is up to date.
[master 19e4405] Adding the first version of split data
 2 files changed, 55 insertions(+), 54 deletions(-)


# Update Version with different seed 


In [116]:
#Updated split of the data into train/validation/test
train_val_data, test_data = train_test_split(data, test_size=0.2, random_state=73)
train_data, validation_data = train_test_split(train_val_data, test_size=0.25, random_state=73)  # 0.25 x 0.8 = 0.2
train_data.to_csv('data/prepared/train.csv', index=False)
validation_data.to_csv('data/prepared/validation.csv', index=False)
test_data.to_csv('data/prepared/test.csv', index=False)

In [41]:
! dvc status
! dvc add "data/prepared"

data/prepared.dvc:                                                              
	changed outs:
		modified:           data/prepared
⠋ Checking graph                                       core>
Adding...                                                                       
!
                                                                                
!
  0% Checking cache in '/home/jexica/Applied-ML-course/Assignment2/.dvc/cache/fi
                                                                                
!
  0%|          |Checking out /home/jexica/Applied-ML-c0/4 [00:00<?,    ?files/s]
 50%|█████     |Checking out /home/jexica/Applied2/4 [00:00<00:00,  9.47files/s]
100% Adding...|████████████████████████████████████████|1/1 [00:00,  3.59file/s]


In [44]:
! git add .
! dvc push

Pushing
!
  0% Querying remote cache|                          |0/1 [00:00<?,    ?files/s]
100% Querying remote cache|█████████████████████|1/1 [00:03<00:00,  3.20s/files]
                                                                                
!
  0% Querying remote cache|                          |0/1 [00:00<?,    ?files/s]
                                                                                
!
  0% Checking cache in '1mBSh692-OHn7R_tIz-j2QxLp6AKN91Jm/files/md5'| |0/? [00:0
  0% Querying cache in '1mBSh692-OHn7R_tIz-j2QxLp6AKN91Jm/files/md5'| |1/256 [00
                                                                                
!
  0% Checking cache in '/home/jexica/Applied-ML-course/Assignment2/.dvc/cache/fi
                                                                                
!
  0%|          |Pushing to gdrive                     0/3 [00:00<?,     ?file/s]
  0%|          |Pushing to gdrive                     0/2 [00:00<?,     ?file/s]

!

  0%| 

In [45]:
! git commit -m "Updated version of split data using different seed"

[master 9d62d02] Updated version of split data using different seed
 1 file changed, 11 insertions(+), 19 deletions(-)


# Target Variable Distribution

In [114]:
def print_label_distribution(data, filename):
    print("Number of 0s in " + filename + " - " + str((data['label']== 0).sum()))
    print("Number of 1s in " + filename + " - " + str((data['label']== 1).sum()))

In [117]:
! git log --oneline

9d62d02 (HEAD, master) Updated version of split data using different seed
32da419 Updated version of split data using different seed
19e4405 Adding the first version of split data
4953743 Updated version of split data using different seed
15d6e38 Adding the first version of split data
28c1430 Updated version of split data using different seed
4b1f95b Adding the first version of split data
7234cc4 Adding the first version of split data
d89e4db Raw data and Initialize dvc


In [130]:
! git checkout 28c1430 "data/prepared.dvc"
! dvc checkout
! dvc pull

Updated 0 paths from 4c0d1fb


Building workspace index                              |8.00 [00:00,  218entry/s]
Comparing indexes                                    |8.00 [00:00, 1.11kentry/s]
Applying changes                                      |0.00 [00:00,     ?file/s]
Fetching
!
  0% Checking cache in '/home/jexica/Applied-ML-course/Assignment2/.dvc/cache/fi
Fetching                                                                        
Building workspace index                             |8.00 [00:00, 1.88kentry/s]
Comparing indexes                                    |8.00 [00:00, 1.97kentry/s]
Applying changes                                      |0.00 [00:00,     ?file/s]
Everything is up to date.


In [123]:
! git log --oneline

9d62d02 (HEAD, master) Updated version of split data using different seed
32da419 Updated version of split data using different seed
19e4405 Adding the first version of split data
4953743 Updated version of split data using different seed
15d6e38 Adding the first version of split data
28c1430 Updated version of split data using different seed
4b1f95b Adding the first version of split data
7234cc4 Adding the first version of split data
d89e4db Raw data and Initialize dvc


In [131]:
train_data = pandas.read_csv('data/prepared/train.csv')
validation_data = pandas.read_csv('data/prepared/validation.csv')
test_data = pandas.read_csv('data/prepared/test.csv')

In [132]:
print_label_distribution(train_data, "train.csv")
print_label_distribution(validation_data, "validation.csv")
print_label_distribution(test_data, "test.csv")

Number of 0s in train.csv - 2624
Number of 1s in train.csv - 812
Number of 0s in validation.csv - 870
Number of 1s in validation.csv - 276
Number of 0s in test.csv - 866
Number of 1s in test.csv - 280
